In [1]:
import warnings
import torch 
warnings.filterwarnings('ignore')
torch.cuda.empty_cache()
torch.cuda.device_count()

3

In [2]:
import os
import torch
from datasets import load_dataset, load_from_disk
from typing import List, Dict
from datasets import Dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from typing import List, Dict

In [3]:
validate_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/data/validate")

In [5]:
model = "tiiuae/falcon-40b-instruct"
cache_dir = "./falcon-cache"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    model_kwargs={'cache_dir':cache_dir}
)

config.json: 100%|██████████| 1.05k/1.05k [00:00<00:00, 2.14MB/s]
config.json: 100%|██████████| 1.05k/1.05k [00:00<00:00, 6.60MB/s]
configuration_falcon.py: 100%|██████████| 7.16k/7.16k [00:00<00:00, 44.3MB/s]
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py: 100%|██████████| 56.9k/56.9k [00:00<00:00, 208MB/s]
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
pytorch_model.bin.index.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 24.9MB/s]
pytorch_model-00001-of-00009.bin:   0%|          | 0.00/9.50G [00:00<?

In [24]:
system_prompt = "You are a helpful, respectful and honest assistant. Below is an instruction that describes a query-focused table summarization task. Write a summary from the table that appropriately response to the user query."

def flatten_table(table: Dict) -> str:
    header = table.get('header', [])
    rows = table.get('rows', [])
    
    flattened_rows = []
    for i, row in enumerate(rows):
        row_text = f"Row {i}, " + ",".join([f"{col}:{val}" for col, val in zip(header, row)])
        flattened_rows.append("## " + row_text)

    flattened_table = " ".join(flattened_rows)
    return flattened_table

task = "Using the information from the table, generate a paragraph-long summary to response to the following user query."

def generate_prompt(example):
    table = example['table']
    title = table['title']
    table = flatten_table(table)
    query = example['query']
    return f"""User: {system_prompt}
    
Table Title: {title}
Table: {table.strip()}
{task}
Query: {query.strip()} \n
Assistant: """.strip()

print(generate_prompt(validate_df[1]))

User: You are a helpful, respectful and honest assistant. Below is an instruction that describes a query-focused table summarization task. Write a summary from the table that appropriately response to the user query.
    
Table Title: Swiss Locomotive And Machine Works
Table: ## Row 0, Built:1895,Number:1,Type:Mountain Railway Rack Steam Locomotive,Slm Number:923,Wheel Arrangement:0 - 4 - 2 T,Location:Snowdon Mountain Railway,Notes:Ladas ## Row 1, Built:1895,Number:2,Type:Mountain Railway Rack Steam Locomotive,Slm Number:924,Wheel Arrangement:0 - 4 - 2 T,Location:Snowdon Mountain Railway,Notes:Enid ## Row 2, Built:1895,Number:3,Type:Mountain Railway Rack Steam Locomotive,Slm Number:925,Wheel Arrangement:0 - 4 - 2 T,Location:Snowdon Mountain Railway,Notes:Wyddfa ## Row 3, Built:1896,Number:4,Type:Mountain Railway Rack Steam Locomotive,Slm Number:988,Wheel Arrangement:0 - 4 - 2 T,Location:Snowdon Mountain Railway,Notes:Snowdon ## Row 4, Built:1896,Number:5,Type:Mountain Railway Rack Stea

In [25]:
prompt = generate_prompt(validate_df[1])

from tqdm import tqdm
for example in tqdm(validate_df):
    prompt = generate_prompt(validate_df[4])
    sequences = pipeline(
        prompt,
        max_length=400,
        return_full_text=False, 
        do_sample=True,
        top_k=20,
        temperature=0.0001,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    print(prompt)
    break

  0%|          | 0/200 [00:01<?, ?it/s]

User: You are a helpful, respectful and honest assistant. Below is an instruction that describes a query-focused table summarization task. Write a summary from the table that appropriately response to the user query.
    
Table Title: Dancemania 8 - Further details
Table: ## Row 0, #:1,Track:Samba De Janeiro,Length:4:05,BPM:133,Ref:,Artist(s):Bellini,From or based in:GermanyGermany,Ref: ## Row 1, #:2,Track:Latin Lover,Length:2:43,BPM:126,Ref:,Artist(s):Popsie,From or based in:SwedenSweden,Ref: ## Row 2, #:3,Track:Stay,Length:2:40,BPM:124,Ref:,Artist(s):Mizz Maya,From or based in:DenmarkDenmark,Ref: ## Row 3, #:4,Track:Hot Hot Hot,Length:2:43,BPM:128,Ref:,Artist(s):Splash,From or based in:United KingdomUnited Kingdom,Ref: ## Row 4, #:5,Track:Wham Bam (Shang-A-Lang),Length:3:12,BPM:132,Ref:,Artist(s):The Scarlet,From or based in:Unknown,Ref:— ## Row 5, #:6,Track:All I Do,Length:2:56,BPM:131,Ref:,Artist(s):XXL,From or based in:GermanyGermany,Ref: ## Row 6, #:7,Track:Let Me Feel the Love,L

In [27]:
sequences

[{'generated_text': ' The'}]